# Juice Toy Problem E

This problem illustrates the use of pricing data in the openLCA database.

## Problem Statement
 
A citrus fruit producer wishes to choose the optimal blend of fruit juice over a 2 day period that minimises cost. The fruit supply options are globally sources lemon, mandarins or oranges. The environmental impact production and the transport of fruit is the same for each fruit.

Over the time horizon of 2 days, the producer is required to deliver 2 kg of blended juice. The cost of producing 1kg of fruit varies  and is shown in the table below.

| Fruit | Day 1 | Day 2 |
| --- | --- | --- |
| Lemon | £1 | £4 |
| Orange | £2 | £3 |
| Mandarin | £3 | £2 |

The cost of storing 1kg of fruit at the blending plant is £1 per day.

## Specification

In this problem, we minimise the cost of blended juice production over a two day period $t\in T=\{1,2\}$. We ignore the environmental impact of production and transport.

The objective is

$$
\min\sum_{f_m, t} Flow_{f_{m}, t}\phi_{f_m, t} + \sum_{f_s, t} S_{f_s, t}\phi_{f_s, t}.
$$

where the cost of producing 1kg of fruit in a day is given by $\phi_{f_m, t}$ in the table above and the cost of storing each fruit is $\phi_{f_s, t}=1$.

There is no initial stored juice.

There total demand is $D^{total}=2kg$ of blended fruit juice so

$$
\sum_{f_m, t} Flow_{f_m, t} \geq D^{total} = 2,
$$

and

$$
S_{f_{s_i}, 2} = Flow_{f_{m_i}, 1}
$$

We also require $Flow_{f_m, t} \geq 0$ and $S_{f_s, t}\geq 0$.

## Pyomo implementation

In [1]:
from pyomo.environ import * 
import pandas as pd

### Create a concrete model

In [2]:
model_C = ConcreteModel()

### Define sets. Will come from mola, but currently hardcoded.

Flows have units kg per day. Time has units of day.

In [3]:
model_C.t = Set(initialize=[1,2], doc='Time period')
model_C.AF = Set(initialize=['Lemon Juice', 'Orange Juice', 'Mandarin Juice'], doc='All flows')
model_C.Fm = Set(initialize=['Lemon Juice', 'Orange Juice', 'Mandarin Juice'], doc='Material flows to optimise')
model_C.Fs = Set(initialize=['Lemon Juice Storage', 'Orange Juice Storage', 'Mandarin Juice Storage'], doc='Service flows to optimise')

### Define parameters.

#### Get cost parameters. Will come from mola, but currently hardcoded.

In [4]:
m = {}
for i, f in enumerate(model_C.Fm):
    m[f, 1] = i + 1
    m[f, 2] = 4 - i
model_C.phimt = Param(model_C.Fm, model_C.t, initialize=m, within=Any, doc='Cost of material flow fm at time t')
model_C.phist = Param(model_C.Fs, model_C.t, initialize=1, within=Any, doc='Cost of material flow fm at time t')
model_C.phimt.pprint()

phimt : Cost of material flow fm at time t
    Size=6, Index=phimt_index, Domain=Any, Default=None, Mutable=False
    Key                   : Value
       ('Lemon Juice', 1) :     1
       ('Lemon Juice', 2) :     4
    ('Mandarin Juice', 1) :     3
    ('Mandarin Juice', 2) :     2
      ('Orange Juice', 1) :     2
      ('Orange Juice', 2) :     3


### Define continuous decision variables

In [5]:
model_C.y = Var(model_C.Fm | model_C.Fs, model_C.t, within=NonNegativeReals, doc='Decision variables')
model_C.y.pprint()

y : Decision variables
    Size=12, Index=y_index
    Key                           : Lower : Value : Upper : Fixed : Stale : Domain
               ('Lemon Juice', 1) :     0 :  None :  None : False :  True : NonNegativeReals
               ('Lemon Juice', 2) :     0 :  None :  None : False :  True : NonNegativeReals
       ('Lemon Juice Storage', 1) :     0 :  None :  None : False :  True : NonNegativeReals
       ('Lemon Juice Storage', 2) :     0 :  None :  None : False :  True : NonNegativeReals
            ('Mandarin Juice', 1) :     0 :  None :  None : False :  True : NonNegativeReals
            ('Mandarin Juice', 2) :     0 :  None :  None : False :  True : NonNegativeReals
    ('Mandarin Juice Storage', 1) :     0 :  None :  None : False :  True : NonNegativeReals
    ('Mandarin Juice Storage', 2) :     0 :  None :  None : False :  True : NonNegativeReals
              ('Orange Juice', 1) :     0 :  None :  None : False :  True : NonNegativeReals
              ('Orange Juice',

### Define constraints

In [6]:
def demand_constraint(model_C):
    s = sum(model_C.y[fm,t] for fm in model_C.Fm for t in model_C.t)
    return s >= 2
model_C.demand_constraint = Constraint(rule=demand_constraint)

model_C.storage_constraint_list = ConstraintList()
model_C.storage_constraint_list.add(model_C.y['Lemon Juice',1] == model_C.y['Lemon Juice Storage',2])
model_C.storage_constraint_list.add(model_C.y['Orange Juice',1] == model_C.y['Orange Juice Storage',2])
model_C.storage_constraint_list.add(model_C.y['Mandarin Juice',1] == model_C.y['Mandarin Juice Storage',2])
model_C.storage_constraint_list.pprint()

storage_constraint_list : Size=3, Index=storage_constraint_list_index, Active=True
    Key : Lower : Body                                              : Upper : Active
      1 :   0.0 :       y[Lemon Juice,1] - y[Lemon Juice Storage,2] :   0.0 :   True
      2 :   0.0 :     y[Orange Juice,1] - y[Orange Juice Storage,2] :   0.0 :   True
      3 :   0.0 : y[Mandarin Juice,1] - y[Mandarin Juice Storage,2] :   0.0 :   True


### Define objective

In [7]:
def objective_rule(model_C):
    obj_material = sum(model_C.y[f,t]*model_C.phimt[f,t] for f in model_C.Fm for t in model_C.t)
    obj_service = sum(model_C.y[f,t]*model_C.phist[f,t] for f in model_C.Fs for t in model_C.t)
    return obj_material+obj_service
model_C.obj = Objective(rule=objective_rule, sense=minimize)
model_C.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : y[Lemon Juice,1] + 4*y[Lemon Juice,2] + 2*y[Orange Juice,1] + 3*y[Orange Juice,2] + 3*y[Mandarin Juice,1] + 2*y[Mandarin Juice,2] + y[Lemon Juice Storage,1] + y[Lemon Juice Storage,2] + y[Orange Juice Storage,1] + y[Orange Juice Storage,2] + y[Mandarin Juice Storage,1] + y[Mandarin Juice Storage,2]


### Apply solver

In [8]:
opt = SolverFactory("glpk")
results = opt.solve(model_C)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4.0
  Upper bound: 4.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 13
  Number of nonzeros: 13
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.020465850830078125
# ----------------------------------------------------------
#   Solution Information
# --------------------------------------

### Process results

In [9]:
model_C.y.display()

y : Decision variables
    Size=12, Index=y_index
    Key                           : Lower : Value : Upper : Fixed : Stale : Domain
               ('Lemon Juice', 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
               ('Lemon Juice', 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
       ('Lemon Juice Storage', 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
       ('Lemon Juice Storage', 2) :     0 :  -0.0 :  None : False : False : NonNegativeReals
            ('Mandarin Juice', 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
            ('Mandarin Juice', 2) :     0 :   2.0 :  None : False : False : NonNegativeReals
    ('Mandarin Juice Storage', 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Mandarin Juice Storage', 2) :     0 :  -0.0 :  None : False : False : NonNegativeReals
              ('Orange Juice', 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
              ('Orange Juice',

In [10]:
model_C.obj.pprint()
model_C.obj.value()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : y[Lemon Juice,1] + 4*y[Lemon Juice,2] + 2*y[Orange Juice,1] + 3*y[Orange Juice,2] + 3*y[Mandarin Juice,1] + 2*y[Mandarin Juice,2] + y[Lemon Juice Storage,1] + y[Lemon Juice Storage,2] + y[Orange Juice Storage,1] + y[Orange Juice Storage,2] + y[Mandarin Juice Storage,1] + y[Mandarin Juice Storage,2]
    deprecated. Use the .expr property getter instead


4.0

# Using openLCA data

In [11]:
import qgrid
#import importlib
import mola.dataview as dv
import mola.dataimport as di
from importlib import reload
reload(dv)

dbconn = di.get_sqlite_connection()

In [12]:
#dfr = dv.get_table(dbconn, 'TBL_EXCHANGES')
#dfr